In [1]:
from picamera.array import PiYUVArray
from picamera import PiCamera

from scipy.signal import find_peaks, butter, filtfilt
from pwm import PWM
import tools
import threading
import ipywidgets as ipw
import time
#import matplotlib.pyplot as plt
import skimage as ski

WIDTH_  = 224
HEIGHT_ = 160
MEDIAN_ = WIDTH_ // 2

# Camera resolution
res = (WIDTH_, HEIGHT_)

# Run a track detection algorithm on a single horizontal line.
# Uses YUV420 image format as the Y component corresponds to image intensity (gray image)
# and thus there is no need to convert from RGB to BW

camera = PiCamera()
        
# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 120

# To filter the noise in the image we use a 3rd order Butterworth filter

# Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
b, a = butter(3, 0.02)

pwm0 = PWM(0)
pwm1 = PWM(1)

pwm0.period = 20000000
pwm1.period = 20000000

pwm0.enable = True
pwm1.enable = True

In [2]:
pwm0.duty_cycle = 1000000
pwm1.duty_cycle = 1600000
time.sleep(1)
pwm1.duty_cycle = 1100000
time.sleep(1)
pwm1.duty_cycle = 2100000
time.sleep(1)
pwm1.duty_cycle = 1600000

In [3]:
#max speed that this algo works
#ucla = 1120000
#sonic = 1250000

UCLA_              =  1138500
USC_               =  1275000
SONIC_             =  1275000

CENTER_LINE_       =  HEIGHT_ // 2
INTERVAL_          =  0.0
CENTER_THRESHOLD_  =  WIDTH_ // 10

STRAIGHT_     =  1600000
RIGHT_        =  1100000
LEFT_         =  2100000
NUM_PREV_VAL_ =  8

killSwitch = ipw.ToggleButton(value=False, description='Shut it down')
#minMaxSlider = ipw.IntRangeSlider(value=[UCLA_/1000,SONIC_/1000], min = 1000, max = 1500,step = 1, description = 'Speed Ranges')

#display(minMaxSlider)
display(killSwitch)

def main_loop():
    rawCapture = PiYUVArray(camera, size=res)
    stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

    #for pwm1
    #duty_cycle

    #      (array([482, 542], dtype=int32), {'peak_heights': array([186.831036  , 185.01286929])})

    numFrames = 0

    #if linePos is 1 then the line is to the right of the car
    #if linePos is -1 then the line is to the left of the car
    #if linePos is 0 either uhoh we're doomed
    #if lost is true then the line is lost
    linePos = 0
    lost = False
    prevSteerDC = pwm1.duty_cycle
    prevSteerDiffArr = []
    prevPeakAvgArr = []
    for i in range(NUM_PREV_VAL_):
        prevSteerDiffArr.append(-1)
        prevPeakAvgArr.append(-1)
    
    for f in stream:
        t = time.time()
        #speeds = update_speeds()
        
        #UCLA_ = speeds[0]
        #SONIC_ = speeds[1]
        
        if killSwitch.value:
            break

        numFrames += 1
        # Get the intensity component of the image (a trick to get black and white images)
        I = f.array[:, :, 0]
        # Reset the buffer for the next image
        rawCapture.truncate(0)
        # Select a horizontal line in the middle of the image
        L = I[CENTER_LINE_, :]
        # Smooth the transitions so we can detect the peaks 
        Lf = filtfilt(b, a, L)
        # Find peaks which are higher than 0.5
        p = find_peaks(Lf, height=128)

        totalPeakAmt = 0
        avgPeakLoc = -520
        numPeaks = 0

        for peak in p[0]:
            numPeaks += 1
            totalPeakAmt += peak

        if numPeaks != 0:
            avgPeakLoc = totalPeakAmt / numPeaks
            if numPeaks == 3:
                print("THIS IS THE BEGINNING OF THE END")

        if avgPeakLoc != -520 :
            if avgPeakLoc < MEDIAN_:
                if lost and linePos == 1:
                    #pwm1.duty_cycle = RIGHT_
                    #continue
                    break
                linePos = -1
            else:
                if lost and linePos == -1:
                    #pwm1.duty_cycle = LEFT_
                    #continue
                    break
                linePos = 1

            pwm1.duty_cycle = int(STRAIGHT_ - ((avgPeakLoc - MEDIAN_)/MEDIAN_)*500000)
            lost = False
        else:
            if linePos == -1:
                pwm1.duty_cycle = LEFT_
            elif linePos == 1:
                pwm1.duty_cycle = RIGHT_
            else:
                pwm1.duty_cycle = STRAIGHT_

            lost = True
            #pwm0.duty_cycle = SONIC_

        if abs(prevSteerDC - STRAIGHT_) < abs(pwm1.duty_cycle - STRAIGHT_) and prevSteerDiffArr[0] > 0 and prevSteerDiffArr[1] > 0 and abs(pwm1.duty_cycle - STRAIGHT_) > 400000:
            pwm0.duty_cycle = SONIC_
        elif abs(avgPeakLoc - MEDIAN_) <= CENTER_THRESHOLD_:
            i = 0
            while i < NUM_PREV_VAL_:
                if abs(prevPeakAvgArr[i] - MEDIAN_) > CENTER_THRESHOLD_:
                    break
                i += 1
            #print("STRAIGHT")
            if i == NUM_PREV_VAL_:
                pwm0.duty_cycle = USC_
        else:
            pwm0.duty_cycle = UCLA_
            if abs(pwm1.duty_cycle - STRAIGHT_) > 300000:
                pwm0.duty_cycle = UCLA_ + 4000

        i = NUM_PREV_VAL_ - 1
        while i > 0:
            prevSteerDiffArr[i] = prevSteerDiffArr[i - 1]
            prevPeakAvgArr[i] = prevPeakAvgArr[i - 1]
            i -= 1
        
        prevSteerDiffArr[0] = abs(pwm1.duty_cycle - STRAIGHT_) - abs(prevSteerDC - STRAIGHT_)
        prevPeakAvgArr[0] = avgPeakLoc

        prevSteerDC = pwm1.duty_cycle

        time_elapsed = time.time() - t
        
        print(prevPeakAvgArr)
        print(prevSteerDiffArr)
        print(sum(prevSteerDiffArr))
        print(sum(prevSteerDiffArr) / len(prevSteerDiffArr))
        print(pwm1.duty_cycle)
        print("Elapsed {:0.4f} seconds, estimated FPS {:0.2f}".format(time_elapsed, 1 / time_elapsed))
        time.sleep(INTERVAL_)
        
    pwm0.duty_cycle = 1000000

    # Release resources
    stream.close()
    rawCapture.close()
    camera.close()

IntRangeSlider(value=(1138, 1275), description='Speed Ranges', max=1500, min=1000)

ToggleButton(value=False, description='Shut it down')

In [5]:
'''
def update_speeds():
    UCLA_ = minMaxSlider.value[0] * 1000
    SONIC_ = minMaxSlider.value[1] * 1000
    return [UCLA_, SONIC_]
minMaxSlider.observe(update_speeds)
'''

In [ ]:
t = threading.Thread(target=main_loop)
t.start()

[-520, -1, -1, -1, -1, -1, -1, -1]
[0, -1, -1, -1, -1, -1, -1, -1]
-7
-0.875
1600000
Elapsed 0.0065 seconds, estimated FPS 153.55
[-520, -520, -1, -1, -1, -1, -1, -1]
[0, 0, -1, -1, -1, -1, -1, -1]
-6
-0.75
1600000
Elapsed 0.0055 seconds, estimated FPS 183.18
[-520, -520, -520, -1, -1, -1, -1, -1]
[0, 0, 0, -1, -1, -1, -1, -1]
-5
-0.625
1600000
Elapsed 0.0063 seconds, estimated FPS 157.87
[-520, -520, -520, -520, -1, -1, -1, -1]
[0, 0, 0, 0, -1, -1, -1, -1]
-4
-0.5
1600000
Elapsed 0.0054 seconds, estimated FPS 183.59
[-520, -520, -520, -520, -520, -1, -1, -1]
[0, 0, 0, 0, 0, -1, -1, -1]
-3
-0.375
1600000
Elapsed 0.0056 seconds, estimated FPS 179.25
[-520, -520, -520, -520, -520, -520, -1, -1]
[0, 0, 0, 0, 0, 0, -1, -1]
-2
-0.25
1600000
Elapsed 0.0056 seconds, estimated FPS 180.08
[-520, -520, -520, -520, -520, -520, -520, -1]
[0, 0, 0, 0, 0, 0, 0, -1]
-1
-0.125
1600000
Elapsed 0.0057 seconds, estimated FPS 175.78
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]

[119.0, 158.0, -520, -520, -520, -520, -520, -520]
[-174108, -294642, 0, 0, 0, 0, 0, 0]
-468750
-58593.75
1568750
Elapsed 0.0053 seconds, estimated FPS 189.20
[66.0, 119.0, 158.0, -520, -520, -520, -520, -520]
[174107, -174108, -294642, 0, 0, 0, 0, 0]
-294643
-36830.375
1805357
Elapsed 0.0058 seconds, estimated FPS 172.16
[6.0, 66.0, 119.0, 158.0, -520, -520, -520, -520]
[267857, 174107, -174108, -294642, 0, 0, 0, 0]
-26786
-3348.25
2073214
Elapsed 0.0061 seconds, estimated FPS 164.21
[5.0, 6.0, 66.0, 119.0, 158.0, -520, -520, -520]
[4464, 267857, 174107, -174108, -294642, 0, 0, 0]
-22322
-2790.25
2077678
Elapsed 0.0057 seconds, estimated FPS 176.11
[-520, 5.0, 6.0, 66.0, 119.0, 158.0, -520, -520]
[22322, 4464, 267857, 174107, -174108, -294642, 0, 0]
0
0.0
2100000
Elapsed 0.0055 seconds, estimated FPS 182.35
[-520, -520, 5.0, 6.0, 66.0, 119.0, 158.0, -520]
[0, 22322, 4464, 267857, 174107, -174108, -294642, 0]
0
0.0
2100000
Elapsed 0.0059 seconds, estimated FPS 169.81
[-520, -520, -520,

[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]
0
0.0
2100000
Elapsed 0.0058 seconds, estimated FPS 172.48
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]
0
0.0
2100000
Elapsed 0.0059 seconds, estimated FPS 170.20
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]
0
0.0
2100000
Elapsed 0.0057 seconds, estimated FPS 174.70
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]
0
0.0
2100000
Elapsed 0.0059 seconds, estimated FPS 169.10
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]
0
0.0
2100000
Elapsed 0.0059 seconds, estimated FPS 170.26
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]
0
0.0
2100000
Elapsed 0.0059 seconds, estimated FPS 168.85
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0, 0]
0
0.0
2100000
Elapsed 0.0059 seconds, estimated FPS 169.38
[-520, -520, -520, -520, -520, -520, -520, -520]
[0, 0, 0, 0, 0, 0, 0

[65.0, 63.0, 58.0, 56.0, 51.0, 46.0, 42.0, 41.0]
[-8929, -22321, -8929, -22321, -22321, -17858, -4464, 22322]
-84821
-10602.625
1809821
Elapsed 0.0058 seconds, estimated FPS 173.59
[64.0, 65.0, 63.0, 58.0, 56.0, 51.0, 46.0, 42.0]
[4464, -8929, -22321, -8929, -22321, -22321, -17858, -4464]
-102679
-12834.875
1814285
Elapsed 0.0058 seconds, estimated FPS 172.33
[64.0, 64.0, 65.0, 63.0, 58.0, 56.0, 51.0, 46.0]
[0, 4464, -8929, -22321, -8929, -22321, -22321, -17858]
-98215
-12276.875
1814285
Elapsed 0.0056 seconds, estimated FPS 179.04
[64.0, 64.0, 64.0, 65.0, 63.0, 58.0, 56.0, 51.0]
[0, 0, 4464, -8929, -22321, -8929, -22321, -22321]
-80357
-10044.625
1814285
Elapsed 0.0056 seconds, estimated FPS 179.61
[63.0, 64.0, 64.0, 64.0, 65.0, 63.0, 58.0, 56.0]
[4465, 0, 0, 4464, -8929, -22321, -8929, -22321]
-53571
-6696.375
1818750
Elapsed 0.0058 seconds, estimated FPS 172.12
[58.0, 63.0, 64.0, 64.0, 64.0, 65.0, 63.0, 58.0]
[22321, 4465, 0, 0, 4464, -8929, -22321, -8929]
-8929
-1116.125
1841071
El

[95.0, 77.0, 61.0, 51.0, 53.0, 65.0, 83.0, 109.0]
[-80358, -71428, -44643, 8929, 53571, 80357, 116072, -62501]
-1
-0.125
1675892
Elapsed 0.0062 seconds, estimated FPS 162.11
[108.0, 95.0, 77.0, 61.0, 51.0, 53.0, 65.0, 83.0]
[-58035, -80358, -71428, -44643, 8929, 53571, 80357, 116072]
4465
558.125
1617857
Elapsed 0.0051 seconds, estimated FPS 195.47
[119.0, 108.0, 95.0, 77.0, 61.0, 51.0, 53.0, 65.0]
[13393, -58035, -80358, -71428, -44643, 8929, 53571, 80357]
-98214
-12276.75
1568750
Elapsed 0.0052 seconds, estimated FPS 191.32
[129.0, 119.0, 108.0, 95.0, 77.0, 61.0, 51.0, 53.0]
[44643, 13393, -58035, -80358, -71428, -44643, 8929, 53571]
-133928
-16741.0
1524107
Elapsed 0.0051 seconds, estimated FPS 195.21
[128.0, 129.0, 119.0, 108.0, 95.0, 77.0, 61.0, 51.0]
[-4464, 44643, 13393, -58035, -80358, -71428, -44643, 8929]
-191963
-23995.375
1528571
Elapsed 0.0051 seconds, estimated FPS 195.07
[127.0, 128.0, 129.0, 119.0, 108.0, 95.0, 77.0, 61.0]
[-4464, -4464, 44643, 13393, -58035, -80358, -7

[74.0, 69.0, 59.0, 56.0, 50.0, 39.0, 38.0, 43.0]
[-22322, -44643, -13393, -26785, -49107, -4465, 22322, 22321]
-116072
-14509.0
1769642
Elapsed 0.0060 seconds, estimated FPS 166.18
[79.0, 74.0, 69.0, 59.0, 56.0, 50.0, 39.0, 38.0]
[-22321, -22322, -44643, -13393, -26785, -49107, -4465, 22322]
-160714
-20089.25
1747321
Elapsed 0.0057 seconds, estimated FPS 176.31
[74.0, 79.0, 74.0, 69.0, 59.0, 56.0, 50.0, 39.0]
[22321, -22321, -22322, -44643, -13393, -26785, -49107, -4465]
-160715
-20089.375
1769642
Elapsed 0.0057 seconds, estimated FPS 175.29
[71.0, 74.0, 79.0, 74.0, 69.0, 59.0, 56.0, 50.0]
[13393, 22321, -22321, -22322, -44643, -13393, -26785, -49107]
-142857
-17857.125
1783035
Elapsed 0.0058 seconds, estimated FPS 172.71
[69.0, 71.0, 74.0, 79.0, 74.0, 69.0, 59.0, 56.0]
[8929, 13393, 22321, -22321, -22322, -44643, -13393, -26785]
-84821
-10602.625
1791964
Elapsed 0.0065 seconds, estimated FPS 153.44
[60.0, 69.0, 71.0, 74.0, 79.0, 74.0, 69.0, 59.0]
[40178, 8929, 13393, 22321, -22321, -2

In [ ]:
'''plt.imshow(I)'''

In [2]:
'''plt.plot(L, label="raw")
plt.plot(Lf, label="filtered")
plt.ylim([0, 300])
plt.legend()'''

NameError: name 'plt' is not defined

In [1]:
camera.close()

NameError: name 'camera' is not defined

In [7]:
pwm0.duty_cycle = 1000000

In [1]:
from pwm import PWM

pwm0 = PWM(0)

pwm0.duty_cycle = 1000000

pwm0.period = 20000000

pwm0.enable = True